In [3]:
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot

from keras.utils import to_categorical
import os
import numpy as np
import tensorflow as tf
import h5py
import math


from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)




In [28]:
#PREPROCESSING 


train_data = h5py.File('datasets/train_signs.h5', "r")
# for n in train_data.keys():
#     print(n)
    
train_set_x = np.array(train_data['train_set_x'][:])
train_set_y = np.array(train_data['train_set_y'][:])

test_data = h5py.File('datasets/test_signs.h5', "r")

test_set_x = np.array(test_data['test_set_x'][:])
test_set_y = np.array(test_data['test_set_y'][:])
classes_all = np.array(test_data["list_classes"][:])


test_y = test_set_y.reshape(1, -1)
train_y = train_set_y.reshape(1, train_set_y.shape[0])



X_test = test_set_x/255.
X_train = train_set_x/255.



# Convert training and test labels to one hot matrices
Y_train = to_categorical(train_y)
Y_test = to_categorical(test_y )




print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))
# print(classes_all.shape[0])


number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)
6


In [16]:
def identity_block(X, f, filters, stage, block):
   
    
    # names
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    
    
    # Second component of main path 
    X = Conv2D(filters = F2, kernel_size=(f,f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base+'2b' )(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size=(1,1), strides =(1,1), padding = 'valid', name = conv_name_base + '2c',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base+'2c' )(X)

    # Add shortcut 
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    
    return X

In [17]:
def convolutional_block(X, f, filters, stage, block, s ):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    
    F1, F2, F3 = filters
    
    
    X_shortcut = X

    X = Conv2D(F1, (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = F2, kernel_size=(f,f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base+'2b' )(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = F3, kernel_size=(1,1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base+ '2c' )(X)
    

    X_shortcut = Conv2D(filters = F3, kernel_size = (1,1), strides = (s,s), padding='valid',name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0) )(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    
 
    
    return X

In [31]:
def Resnet(input_shape , classes):
    
    
    
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((3,3))(X_input)
    # 1
    X = Conv2D(64, (7,7), strides = (2,2,), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides = (2,2))(X)
    
    # 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block = 'a',s = 1)
    X = identity_block(X, 3, filters = [64, 64, 256], stage = 2, block= 'b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    

    # 3 
    X = convolutional_block(X, f = 3, filters = [128,128,512], stage = 3, block = 'a',s = 2)
    X = identity_block(X, 3, filters = [128,128,512], stage = 3, block= 'b')
    X = identity_block(X, 3, filters = [128,128,512], stage = 3, block= 'c')
    X = identity_block(X, 3, filters = [128,128,512], stage = 3, block= 'd')


    # 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'a',s = 2)
    X = identity_block(X, 3, filters = [256, 256, 1024], stage = 4, block= 'b')
    X = identity_block(X, 3, filters = [256, 256, 1024], stage = 4, block= 'c')
    X = identity_block(X, 3, filters = [256, 256, 1024], stage = 4, block= 'd')
    X = identity_block(X, 3, filters = [256, 256, 1024], stage = 4, block= 'e')
    X = identity_block(X, 3, filters = [256, 256, 1024], stage = 4, block= 'f')
    
    # 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block = 'a',s = 2)
    X = identity_block(X, 3, filters = [512, 512, 2048], stage = 5, block= 'b')
    X = identity_block(X, 3, filters = [512, 512, 2048], stage = 5, block= 'c')

    
    X = AveragePooling2D((2,2), name="avg_pool")(X)
    
    

    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # model creation
    model = Model(inputs = X_input, outputs = X, name='Resnet')

    return model

In [32]:
model = Resnet(input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3]), classes = classes_all.shape[0])

In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
model.fit(X_train, Y_train, epochs = 2, batch_size = 32)

Epoch 1/2
1080/1080 [==============================] - 242s - loss: 2.8061 - acc: 0.2657   
Epoch 2/2
1080/1080 [==============================] - 239s - loss: 1.8464 - acc: 0.4157   


In [35]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

120/120 [==============================] - 9s     
Loss = 2.6869026343
Test Accuracy = 0.166666666667
